# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,12.63,72,79,5.11,NZ,1673456986
1,1,cape town,-33.9258,18.4232,24.50,67,96,5.36,ZA,1673456703
2,2,ushuaia,-54.8000,-68.3000,10.81,71,75,7.72,AR,1673456986
3,3,norfolk,42.1668,-71.2495,2.54,68,40,4.12,US,1673456987
4,4,grand gaube,-20.0064,57.6608,26.32,72,67,2.24,MU,1673456987


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.50,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>21) &(city_data_df["Max Temp"]<27),:]


# Display sample data
ideal_weather_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,cape town,-33.9258,18.4232,24.50,67,96,5.36,ZA,1673456703
4,4,grand gaube,-20.0064,57.6608,26.32,72,67,2.24,MU,1673456987
7,7,port alfred,-33.5906,26.8910,24.54,89,97,4.44,ZA,1673456989
12,12,hermanus,-34.4187,19.2345,23.78,78,93,3.64,ZA,1673457020
14,14,atuona,-9.8000,-139.0333,25.52,74,7,5.41,PF,1673457021


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,cape town,ZA,-33.9258,18.4232,67,
4,grand gaube,MU,-20.0064,57.6608,72,
7,port alfred,ZA,-33.5906,26.8910,89,
12,hermanus,ZA,-34.4187,19.2345,78,
14,atuona,PF,-9.8000,-139.0333,74,
...,...,...,...,...,...,...
540,lumut,MY,4.2323,100.6298,86,
547,yeppoon,AU,-23.1333,150.7333,78,
551,manjeshwar,IN,12.7194,74.8861,75,
555,moche,PE,-8.1711,-79.0092,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

#Set the category to serach through

categories ="accommodation.hotel"

#Set the radius to look in

radius = 10000

#Set the complete search paramaters

params = {
    "categories": categories,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: Townhouse Hotel
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
port alfred - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
atuona - nearest hotel: Pearl Resort
hilo - nearest hotel: Dolphin Bay Hotel
plettenberg bay - nearest hotel: Milkwood Manor
garowe - nearest hotel: Red Sea
bambanglipuro - nearest hotel: Puri Ganjuran
antofagasta - nearest hotel: Hotel Colón
bredasdorp - nearest hotel: Victoria Hotel
sucua - nearest hotel: HOTEL BARZA
cockburn town - nearest hotel: The Salt Raker Inn
bengkulu - nearest hotel: Dena Hotel
hithadhoo - nearest hotel: Pebbles Inn
watsa - nearest hotel: No hotel found
bambous virieux - nearest hotel: Otentic Eco Tent Experience
samarai - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
bangassou - nearest hotel: Hotel De Tourisme
bara - nearest hotel: No hotel found
kruisfont

,City,Country,Lat,Lng,Humidity,Hotel Name
1,cape town,ZA,-33.9258,18.4232,67,Townhouse Hotel
4,grand gaube,MU,-20.0064,57.6608,72,Hotel Lux- Grand Gaube
7,port alfred,ZA,-33.5906,26.8910,89,No hotel found
12,hermanus,ZA,-34.4187,19.2345,78,Aloe guest house
14,atuona,PF,-9.8000,-139.0333,74,Pearl Resort
...,...,...,...,...,...,...
540,lumut,MY,4.2323,100.6298,86,The Orient Star Resort Lumut
547,yeppoon,AU,-23.1333,150.7333,78,Blossom's on Seaspray
551,manjeshwar,IN,12.7194,74.8861,75,No hotel found
555,moche,PE,-8.1711,-79.0092,73,Hotel Golf y Country Club


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Hotel Name","Country"],
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)